# The Project: Model Context Protocol (MCP) Demo

This project demonstrates how a local AI agent can communicate with a small **Model Context Protocol (MCP)** server to retrieve relevant context and reason about it — all running entirely inside Google Colab, with no external APIs.

## How It Works

1. **MCP Context Server**
   - A simple FastAPI server listens for POST requests at `/mcp/context`.
   - When the agent sends a query (for example, "analyze latest CVE vulnerabilities" or "patient symptoms"),  
     the server returns a structured context object that includes:
     - The topic (e.g., cybersecurity, health)
     - A list of related documents
     - A short summary

2. **Local Reasoning Agent**
   - A lightweight SentenceTransformer model (`all-MiniLM-L6-v2`) runs locally.
   - It compares the query and the documents using **cosine similarity** to find which document is most relevant.
   - The agent then prints a short explanation describing its reasoning.

3. **End-to-End Flow**
   - The agent sends a query → MCP returns structured context →  
     the agent interprets it → prints the reasoning result.

## Example Output

For a query such as **“patient shows low mood and sleep problems”**,  
the system outputs something like:

=== Retrieved Context ===
{
"topic": "health",
"related_docs": [
"Symptoms: fatigue, sleep issues, low mood",
"Possible diagnosis: mild depression"
],
"summary": "Patient shows consistent mood-related symptoms."
}

=== Local Explanation ===
Based on context topic 'health', the agent found that:
-> Patient shows consistent mood-related symptoms.
-> Most relevant info: Symptoms: fatigue, sleep issues, low mood


## Key Points

- Runs fully locally inside Colab (no ngrok required).
- Demonstrates **MCP communication** and **context-aware reasoning**.
- Simple and modular — can be extended to real data sources like NVD (cybersecurity) or PubMed (medical).


In [1]:
# Step 1: Install dependencies
!pip install -q fastapi uvicorn pyngrok nest_asyncio requests sentence-transformers

Step 2 – Define the MCP context server

In [ ]:
# Load ngrok token from local file "env_ngrok"
# The file should contain a line: NGROK_AUTH_TOKEN=your_token_here
load_dotenv("env_ngrok")
ngrok_token = os.getenv("NGROK_AUTH_TOKEN")

In [ ]:
# Kill any previous Uvicorn processes that are still bound to port 8000
import os
os.system("fuser -k 8000/tcp || echo 'no previous process'")

In [1]:
# ============================================================
# MCP Demo – Context-Aware Local AI Agent
# ============================================================

# ------------------------------------------------------------
# Imports
# ------------------------------------------------------------
import nest_asyncio
import threading
import requests
from fastapi import FastAPI, Request
import uvicorn
from sentence_transformers import SentenceTransformer, util


# ------------------------------------------------------------
# Define the MCP Context Server
# ------------------------------------------------------------
app = FastAPI()

@app.post("/mcp/context")
async def get_context(request: Request):
    """Return structured context information based on the query"""
    data = await request.json()
    query = data.get("query", "").lower()

    if "cve" in query or "vulnerability" in query:
        context = {
            "topic": "cybersecurity",
            "related_docs": [
                "CVE-2025-1234: Privilege escalation in kernel module",
                "CVE-2024-8791: SQL injection in login endpoint"
            ],
            "summary": "Recent critical vulnerabilities require patching in web systems."
        }
    elif "patient" in query or "symptom" in query:
        context = {
            "topic": "health",
            "related_docs": [
                "Symptoms: fatigue, sleep issues, low mood",
                "Possible diagnosis: mild depression"
            ],
            "summary": "Patient shows consistent mood-related symptoms."
        }
    else:
        context = {
            "topic": "general",
            "related_docs": ["No domain-specific context found."],
            "summary": "Query not recognized; generic context returned."
        }

    return {"query": query, "context": context}

# ------------------------------------------------------------
# Start the FastAPI Server in Background (Colab-safe)
# ------------------------------------------------------------
nest_asyncio.apply()

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="error")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# ------------------------------------------------------------
# Define Local Agent Logic
# ------------------------------------------------------------
MCP_URL = "http://127.0.0.1:8000"
model = SentenceTransformer("all-MiniLM-L6-v2")

def get_mcp_context(query):
    """Send query to the MCP server and return context"""
    payload = {"query": query}
    resp = requests.post(MCP_URL + "/mcp/context", json=payload)
    return resp.json()["context"]

def local_reasoner(query):
    """Simulate a reasoning agent that uses context from MCP"""
    ctx = get_mcp_context(query)
    docs = ctx["related_docs"]
    embeddings = model.encode([query] + docs, convert_to_tensor=True)
    sim = util.cos_sim(embeddings[0], embeddings[1:])
    best_idx = int(sim.argmax())

    print("\n=== Query ===")
    print(query)
    print("\n=== Retrieved Context ===")
    print(ctx)
    print("\n=== Best Matched Document ===")
    print(docs[best_idx])
    print("\n=== Local Explanation ===")
    print(f"Based on context topic '{ctx['topic']}', the agent found that:")
    print(f"-> {ctx['summary']}")
    print(f"-> Most relevant info: {docs[best_idx]}")

# ------------------------------------------------------------
# Example Runs
# ------------------------------------------------------------
local_reasoner("analyze latest CVE vulnerabilities")
local_reasoner("patient shows low mood and sleep problems")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



=== Query ===
analyze latest CVE vulnerabilities

=== Retrieved Context ===
{'topic': 'cybersecurity', 'related_docs': ['CVE-2025-1234: Privilege escalation in kernel module', 'CVE-2024-8791: SQL injection in login endpoint'], 'summary': 'Recent critical vulnerabilities require patching in web systems.'}

=== Best Matched Document ===
CVE-2025-1234: Privilege escalation in kernel module

=== Local Explanation ===
Based on context topic 'cybersecurity', the agent found that:
-> Recent critical vulnerabilities require patching in web systems.
-> Most relevant info: CVE-2025-1234: Privilege escalation in kernel module

=== Query ===
patient shows low mood and sleep problems

=== Retrieved Context ===
{'topic': 'health', 'related_docs': ['Symptoms: fatigue, sleep issues, low mood', 'Possible diagnosis: mild depression'], 'summary': 'Patient shows consistent mood-related symptoms.'}

=== Best Matched Document ===
Symptoms: fatigue, sleep issues, low mood

=== Local Explanation ===
Based on